In [10]:
import pandas as pd  # Utilizada para ler e manipular dados em planilhas Excel (DataFrames).
from docx import Document  # Permite criar e manipular documentos do Word (.docx).
from docx.shared import Pt  # Usado para definir o tamanho da fonte nos documentos Word.
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT  # Fornece opções de alinhamento de parágrafos no Word (ex: centralizado).
import warnings  # Usado para ignorar ou tratar mensagens de aviso durante a execução do código.
import re  # Biblioteca padrão para operações com expressões regulares (ex: validações ou extrações de texto).
import win32com.client  # Permite controlar aplicativos do Microsoft Office via automação COM (ex: Word, Excel).
import comtypes.client  # Utilizado especificamente para automação do Word no Windows, incluindo conversão para PDF.
import os  # Fornece funcionalidades para manipulação de arquivos e diretórios no sistema operacional.


In [11]:
"""
1- Inserir o processo de loop para os casos em que a planilha tiver mais de 1 registro ok
2- Converter o resultado final para PDF ok
3- Colocar esse projeto no HB
4- Exportar um histórico para o result

"""

'\n1- Inserir o processo de loop para os casos em que a planilha tiver mais de 1 registro ok\n2- Converter o resultado final para PDF ok\n3- Colocar esse projeto no HB\n4- Exportar um histórico para o result\n\n'

In [12]:
#Criação das funções para a formatação dos dados e conversão para PDF

def formatar_cpf(cpf):
    cpf = ''.join(filter(str.isdigit, str(cpf)))  # Remove quaisquer caracteres não numéricos
    if len(cpf) == 11:
        return f"{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}"
    else:
        return "CPF inválido"

def formatar_telefone(numero):
    # Trata valores None ou NaN
    if numero is None:
        return ""

    # Se for bytes, converte para string
    if isinstance(numero, bytes):
        numero = numero.decode('utf-8')

    # Garante que é uma string
    numero = str(numero)

    # Remove caracteres não numéricos
    numero = re.sub(r'\D', '', numero)

    # Formata com base na quantidade de dígitos
    if len(numero) == 11:
        return f"({numero[:2]}) {numero[2:7]}-{numero[7:]}"
    elif len(numero) == 10:
        return f"({numero[:2]}) {numero[2:6]}-{numero[6:]}"
    else:
        return numero  # Retorna como está se não tiver formato esperado
        
# Função para converter DOCX para PDF (Windows)
def docx_para_pdf(docx_file, pdf_file):
    try:
        word = comtypes.client.CreateObject('Word.Application')
        doc = word.Documents.Open(docx_file)
        doc.SaveAs(pdf_file, FileFormat=17)  # O formato 17 é para PDF
        doc.Close()
        word.Quit()
        print(f"Arquivo PDF salvo com sucesso: {pdf_file}")
    except Exception as e:
        print(f"Erro ao converter para PDF: {e}")

# Função para verificar se o arquivo foi salvo corretamente
def verificar_arquivo(arquivo):
    if os.path.exists(arquivo):
        return True
    else:
        print(f"Arquivo não encontrado: {arquivo}")
        return False

In [13]:
# Ignorar mensagens de aviso
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Carrega os dados da planilha
df = pd.read_excel("Procuração.xlsx")

# Itera sobre cada linha da planilha
for index, dados in df.iterrows():
    # Criação dos campos formatados
    dados['CPF Formatado'] = formatar_cpf(dados['CPF'])
    dados['Telefone Formatado'] = formatar_telefone(dados['Telefone para contato'])

    # Monta o texto com base nas informações
    texto = f"""{dados['Nome Completo']}, {dados['Nacionalidade']}, {dados['Estado Civil']}, {dados['Profissão']}, telefone n° {dados['Telefone Formatado']}, \
com {dados['Tipo do Documento']} n° {dados['Número do Documento']}, inscrito sob o C.P.F. n° {dados['CPF Formatado']}, \
E-mail: {dados['E-mail']}, residente e domiciliado a {dados['Endereço Completo']}, {dados['Bairro']}, {dados['Cidade']}, {dados['Estado']}."""

    # Abre o modelo
    doc = Document('Procuração Modelo.docx')

    # Adiciona o texto principal ao primeiro parágrafo
    first_paragraph = doc.paragraphs[0]
    run = first_paragraph.add_run(texto)
    run.font.name = 'Times New Roman'
    run.font.size = Pt(13)

    # Adiciona um parágrafo com assinatura centralizada
    #Colocar condicional para diferenciar o tipo de documento
    paragrafo = doc.add_paragraph()
    run = paragrafo.add_run(f"__________________________________________________\n{dados['Nome Completo']}\n{dados['CPF Formatado']}")
    paragrafo.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
    run.font.name = 'Times New Roman'
    run.font.size = Pt(13)

    # Salva o documento como arquivo temporário
    nome_arquivo_temp_docx = f"Procuração Temp - {dados['Nome Completo']}.docx"
    nome_arquivo_temp_docx = os.path.abspath(nome_arquivo_temp_docx)  # Garantir o caminho absoluto
    doc.save(nome_arquivo_temp_docx)

    # Caminho para salvar o PDF
    nome_arquivo_pdf = f"Procuração - {dados['Nome Completo']}.pdf"
    nome_arquivo_pdf = os.path.abspath(nome_arquivo_pdf)  # Garantir o caminho absoluto

    # Verifica se o arquivo temporário foi salvo corretamente antes da conversão
    if verificar_arquivo(nome_arquivo_temp_docx):
        # Converte o arquivo DOCX para PDF
        docx_para_pdf(nome_arquivo_temp_docx, nome_arquivo_pdf)

        # Após conversão, exclui o arquivo temporário DOCX
        os.remove(nome_arquivo_temp_docx)  # Remove o arquivo DOCX temporário


Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 2.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 3.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 4.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 5.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 6.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 7.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 8.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 9.pdf
Arquivo PDF salvo com sucesso: C:\Users\lucas\Script Python\Procuração - Pessoa Exemplo 10.pdf


In [14]:
print(df)

       Nome Completo Nacionalidade   Estado Civil   Profissão  \
0   Pessoa Exemplo 2    Brasileira  Divorciado(a)    Analista   
1   Pessoa Exemplo 3    Brasileira  Divorciado(a)      Médico   
2   Pessoa Exemplo 4    Brasileira  Divorciado(a)  Engenheiro   
3   Pessoa Exemplo 5    Brasileira    Solteiro(a)   Professor   
4   Pessoa Exemplo 6    Brasileira      Casado(a)  Engenheiro   
5   Pessoa Exemplo 7    Brasileira      Casado(a)   Professor   
6   Pessoa Exemplo 8    Brasileira  Divorciado(a)    Analista   
7   Pessoa Exemplo 9    Brasileira  Divorciado(a)  Engenheiro   
8  Pessoa Exemplo 10    Brasileira  Divorciado(a)      Médico   

  Tipo do Documento  Número do Documento          CPF  Telefone para contato  \
0                RG              8735921  10897819107             2199287675   
1                RG              7399244  43718088858             2199319046   
2                RG              8191540  73748576914             2199452854   
3                RG          